In [1]:
from scwidgets import (AnswerRegistry, Answer, TextareaAnswer)
from ipywidgets import Layout, Checkbox, VBox

This notebook demonstrates the usage and the functionalities of the `AnswerRegistry` widget, which can be used to provide a notebook with an interface to save and recover answers to questions and exercises, entered into text boxes or into widgets that have been made to inherit from the `Answer` interface.

# User guide

The `AnswerRegistry` widget allows to associate a JSON file (composed from the prefix defined in the code, and a name provided by the user) with the current notebook. One or more widgets will then be registered, so that the corresponding answers can be saved, or loaded.

In [2]:
answer_registry = AnswerRegistry(prefix="demo-answers")
display(answer_registry)

AnswerRegistry(children=(Output(layout=Layout(height='100%', width='100%')), HBox(children=(Dropdown(descripti…

A simple example of the usage of the answer registry is given here: the `TextareaAnswer` widged is simply a `ipywidgets.Textarea` instrumented with the `Answer` interface. It can be registered so that the value of the textarea gets saved into the JSON file, or loaded for checking and/or continuing the exercise session.

`register_answer_widget(key, answer)` takes one string key (that will be used as an entry in the JSON and must uniquely identify the widget) and an `Answer` widget. 

In [3]:
text_answer = TextareaAnswer(value='Write here your answer', layout=Layout(width='99%'))
answer_registry.register_answer_widget("text_answer-demo", text_answer)
display(text_answer)

TextareaAnswer(children=(Textarea(value='Write here your answer', layout=Layout(width='99%')), VBox(children=(…

In every notebook one can register many different answers, and each can be saved individually into the JSON file.

In [4]:
text_answer_other = TextareaAnswer(value='Another answer', layout=Layout(width='99%'))
answer_registry.register_answer_widget("text_answer-demo_other", text_answer_other)
display(text_answer_other)

TextareaAnswer(children=(Textarea(value='Another answer', layout=Layout(width='99%')), VBox(children=(Button(d…

# Developer guide

Any widget can be instrumented to become an answer. It is sufficient to inherit from the `Answer` class, and to implement an `answer_value` property (essentially providing a standardized name for whatever component of the original widget we want to store) and implementing an `on_save` function to handle the display of the save button.

In [5]:
class CheckboxAnswer(VBox, Answer):
    """ A widget that contains a Checkbox whose status can be saved"""
    def __init__(self, *args, **kwargs):
        self._answer_checkbox = Checkbox(*args, **kwargs)
        super(CheckboxAnswer, self).__init__(
                [self._answer_checkbox], layout=Layout(align_items="flex-start", width='100%'))

    
    @property
    def answer_value(self):
        return self._answer_checkbox.value

    @answer_value.setter
    def answer_value(self, new_answer_value):
        self._answer_checkbox.value = new_answer_value

    def on_save(self, callback):
        if self._save_button is None and self._on_save_callback is None:
            save_widget = self._init_save_widget(callback)
            self.children += (save_widget,)
        else:
            self._answer_checkbox(callback)

Does the barber who who shaves all those, and those only, who do not shave themselves shave himself?

In [6]:
check_answer = CheckboxAnswer(description="The barber shaves himself", value=False)
answer_registry.register_answer_widget("check_answer-barber", check_answer)
display(check_answer)

CheckboxAnswer(children=(Checkbox(value=False, description='The barber shaves himself'), VBox(children=(Button…

## TODO

* add a on-change mechanism to signal when the answer has been changed and it must be saved